In [34]:
import pandas as pd
import scipy.stats as stats
from scipy.stats import kruskal
import plotly.express as px
import plotly.graph_objects as go
from sklearn.utils import resample
from itertools import combinations
from lifelines.statistics import logrank_test

from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.statistics import multivariate_logrank_test
import plotly.io as pio
pio.renderers.default='notebook'

In [2]:
bd6 = pd.read_csv("./Datos/mortalidad_semana2.csv")
bd6_V_farnesiana = bd6.loc[bd6['Especie'] == 'V. farnesiana']
bd6_R_communis = bd6.loc[bd6['Especie'] == 'R. communis']

In [3]:
bd6_V_farnesiana.head(5)

,Especie,Tratamiento,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10,H11,H12,H13,H14,H15,Time,status
0,V. farnesiana,C,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,15,0
1,V. farnesiana,C,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,15,0
2,V. farnesiana,C,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1
3,V. farnesiana,C,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,15,0
9,V. farnesiana,C,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,15,0


In [4]:
n_iterations = 1000
new_data_df = pd.DataFrame()

tratamientos = ["C","CI","J","JI","SN"]

for tratamiento in tratamientos:
    mi_db = bd6_V_farnesiana.loc[bd6_V_farnesiana['Tratamiento'] == tratamiento].copy().drop(["Tratamiento","Especie","status"],axis=1)
    new_data_df = pd.DataFrame()
    #Boostrap
    for i in range(n_iterations):
        newData = resample(mi_db, replace=True, n_samples=len(mi_db))
        new_data_df = new_data_df.append(newData.mean().to_frame().T, ignore_index = True)
    new_data_df = new_data_df.append(mi_db, ignore_index = True)
    new_data_df.loc[new_data_df['Time'] == 15, 'status'] = '0' 
    new_data_df.loc[new_data_df['Time'] < 15, 'status'] = '1' 

In [23]:
import pandas as pd
import scipy.stats as stats

# Supongamos que tenemos una columna llamada "data" en un DataFrame llamado "df"
# y queremos calcular el intervalo de confianza del 95% para esa columna.

# Calcular la media y la desviación estándar de la columna
mean = new_data_df['Time'].mean()
std = new_data_df['Time'].std()

# Calcular el tamaño de la muestra
n = len(new_data_df['Time'])

# Calcular el error estándar
se = std / (n ** 0.5)

# Calcular el valor crítico de Z para un nivel de confianza del 95%
z_critical = stats.norm.ppf(0.975)

# Calcular los límites del intervalo de confianza
ci_lower = mean - z_critical * se
ci_upper = mean + z_critical * se

# Imprimir el resultado
print("Intervalo de confianza del 95%: ({:.4f}, {:.4f})".format(ci_lower, ci_upper))


Intervalo de confianza del 95%: (15.0000, 15.0000)


In [18]:
new_data_df["Time"]
# create 99% confidence interval
sct.t.interval(alpha=0.99,
              df=new_data_df["Time"],
              loc=np.mean(new_data_df["Time"]), 
              scale=sct.sem(new_data_df["Time"]))

(array([nan, nan, nan, ..., nan, nan, nan]),
 array([nan, nan, nan, ..., nan, nan, nan]))

In [85]:
n_iterations = 1000
new_data_df = pd.DataFrame()
fig = go.Figure()
tratamientos = ["C","CI","J","JI","SN"]
LogRankTratamientos = []
for tratamiento in tratamientos:
    mi_db = bd6_V_farnesiana.loc[bd6_V_farnesiana['Tratamiento'] == tratamiento].copy().drop(["Tratamiento","Especie","status"],axis=1)
    new_data_df = pd.DataFrame()
    #Boostrap
    for i in range(n_iterations):
        newData = resample(mi_db, replace=True, n_samples=len(mi_db))
        new_data_df = new_data_df.append(newData.mean().to_frame().T, ignore_index = True)
    new_data_df = new_data_df.append(mi_db, ignore_index = True)
    new_data_df.loc[new_data_df['Time'] == 15, 'status'] = 0 
    new_data_df.loc[new_data_df['Time'] < 15, 'status'] = 1

    # Ajustar el modelo de Kaplan-Meier
    kmf = KaplanMeierFitter()
    kmf.fit(new_data_df['Time'], new_data_df['status'],label = "{}".format(tratamiento))
    LogRankTratamientos.append([new_data_df['Time'], new_data_df['status']])
    # Imprimir la tabla de supervivencia
    #kmf.plot_survival_function(show_censors=True)
    #fig = px.line(kmf.survival_function_,title="Alturas H15")

    #fig.show()
    #print(kmf.survival_function_)
    fig.add_trace(go.Scatter(x=kmf.survival_function_.index, y=kmf.survival_function_["{}".format(tratamiento)],
                    mode='lines+markers',
                    name="{}".format(tratamiento)))
    #print(mi_db)

    # Calcular la media y la desviación estándar de la columna
    mean = new_data_df['Time'].mean()
    std = new_data_df['Time'].std()

    # Calcular el tamaño de la muestra
    n = len(new_data_df['Time'])

    # Calcular el error estándar
    se = std / (n ** 0.5)

    # Calcular el valor crítico de Z para un nivel de confianza del 95%
    z_critical = stats.norm.ppf(0.975)

    # Calcular los límites del intervalo de confianza
    ci_lower = mean - z_critical * se
    ci_upper = mean + z_critical * se

    # Imprimir el resultado
    print("Intervalo de confianza del tratamiento {} del 95%: ({:.4f}, {:.4f}) y media del {}".format(tratamiento,ci_lower, ci_upper,mean))

fig.update_layout(title='Prueba de super vivencia Kaplan-Meier')
fig.update_yaxes(
    range=(0, 1.1),
    constrain='domain'
)
fig.show()

Intervalo de confianza del tratamiento C del 95%: (13.5913, 13.7546) y media del 13.672970297029702
Intervalo de confianza del tratamiento CI del 95%: (13.8996, 14.0311) y media del 13.965346534653467
Intervalo de confianza del tratamiento J del 95%: (9.9929, 10.2491) y media del 10.120990099009902
Intervalo de confianza del tratamiento JI del 95%: (7.2594, 7.5130) y media del 7.386237623762376
Intervalo de confianza del tratamiento SN del 95%: (15.0000, 15.0000) y media del 15.0


In [86]:
tratamientos = ["C", "CI", "J", "JI", "SN"]

# Obtener todas las combinaciones de máximo 2 elementos
combinaciones = list(combinations(LogRankTratamientos, 2))
combinaciones2 = list(combinations(tratamientos, 2))

# Calcular el log-rank test para cada combinación
basura = 0
for combinacion in combinaciones:
    time1, status1 = combinacion[0]
    time2, status2 = combinacion[1]
    result = logrank_test(time1, status1, time2, status2)
    
    # Imprimir los resultados

    valor = combinaciones2[basura]

    print("Log-Rank Test para la combinación", valor)
    print("Estadístico:", result.test_statistic)
    print("Valor p:", result.p_value)
    print()
    basura = basura + 1

Log-Rank Test para la combinación ('C', 'CI')
Estadístico: 1076.033828195712
Valor p: 5.3437663699929565e-236

Log-Rank Test para la combinación ('C', 'J')
Estadístico: 1935.162576305829
Valor p: 0.0

Log-Rank Test para la combinación ('C', 'JI')
Estadístico: 1954.8273468655955
Valor p: 0.0

Log-Rank Test para la combinación ('C', 'SN')
Estadístico: 0.0
Valor p: 1.0

Log-Rank Test para la combinación ('CI', 'J')
Estadístico: 1940.9429070405913
Valor p: 0.0

Log-Rank Test para la combinación ('CI', 'JI')
Estadístico: 1962.9714787706075
Valor p: 0.0

Log-Rank Test para la combinación ('CI', 'SN')
Estadístico: 0.0
Valor p: 1.0

Log-Rank Test para la combinación ('J', 'JI')
Estadístico: 1989.9190761002308
Valor p: 0.0

Log-Rank Test para la combinación ('J', 'SN')
Estadístico: 0.0
Valor p: 1.0

Log-Rank Test para la combinación ('JI', 'SN')
Estadístico: 0.0
Valor p: 1.0



In [80]:
n_iterations = 1000
new_data_df = pd.DataFrame()
fig = go.Figure()
tratamientos = ["C","CI","J","JI","SN"]
LogRankTratamientos = []
for tratamiento in tratamientos:
    mi_db = bd6_R_communis.loc[bd6_R_communis['Tratamiento'] == tratamiento].copy().drop(["Tratamiento","Especie","status"],axis=1)
    new_data_df = pd.DataFrame()
    #Boostrap
    for i in range(n_iterations):
        newData = resample(mi_db, replace=True, n_samples=len(mi_db))
        new_data_df = new_data_df.append(newData.mean().to_frame().T, ignore_index = True)
    new_data_df = new_data_df.append(mi_db, ignore_index = True)
    new_data_df.loc[new_data_df['Time'] == 15, 'status'] = 0 
    new_data_df.loc[new_data_df['Time'] < 15, 'status'] = 1

    # Ajustar el modelo de Kaplan-Meier
    kmf = KaplanMeierFitter()
    kmf.fit(new_data_df['Time'], new_data_df['status'],label = "{}".format(tratamiento))
    LogRankTratamientos.append([new_data_df['Time'], new_data_df['status']])
    # Imprimir la tabla de supervivencia
    #kmf.plot_survival_function(show_censors=True)
    #fig = px.line(kmf.survival_function_,title="Alturas H15")

    #fig.show()
    #print(kmf.survival_function_)
    fig.add_trace(go.Scatter(x=kmf.survival_function_.index, y=kmf.survival_function_["{}".format(tratamiento)],
                    mode='lines+markers',
                    name="{}".format(tratamiento)))
    #print(mi_db)

    # Calcular la media y la desviación estándar de la columna
    mean = new_data_df['Time'].mean()
    std = new_data_df['Time'].std()

    # Calcular el tamaño de la muestra
    n = len(new_data_df['Time'])

    # Calcular el error estándar
    se = std / (n ** 0.5)

    # Calcular el valor crítico de Z para un nivel de confianza del 95%
    z_critical = stats.norm.ppf(0.975)

    # Calcular los límites del intervalo de confianza
    ci_lower = mean - z_critical * se
    ci_upper = mean + z_critical * se

    # Imprimir el resultado
    print("Intervalo de confianza del tratamiento {} del 95%: ({:.4f}, {:.4f}) y media del {}".format(tratamiento,ci_lower, ci_upper,mean))

fig.update_layout(title='Prueba de super vivencia Kaplan-Meier')
fig.update_yaxes(
    range=(0, 1.1),
    constrain='domain'
)
fig.show()

Intervalo de confianza del tratamiento C del 95%: (12.1851, 12.4066) y media del 12.295841584158415
Intervalo de confianza del tratamiento CI del 95%: (12.7474, 12.9251) y media del 12.836237623762376
Intervalo de confianza del tratamiento J del 95%: (6.8689, 7.0975) y media del 6.983168316831684
Intervalo de confianza del tratamiento JI del 95%: (6.7096, 6.9637) y media del 6.836633663366337
Intervalo de confianza del tratamiento SN del 95%: (13.8276, 13.9680) y media del 13.89782178217822


In [81]:
tratamientos = ["C", "CI", "J", "JI", "SN"]

# Obtener todas las combinaciones de máximo 2 elementos
combinaciones = list(combinations(LogRankTratamientos, 2))
combinaciones2 = list(combinations(tratamientos, 2))

# Calcular el log-rank test para cada combinación
basura = 0
for combinacion in combinaciones:
    time1, status1 = combinacion[0]
    time2, status2 = combinacion[1]
    result = logrank_test(time1, status1, time2, status2)
    
    # Imprimir los resultados

    valor = combinaciones2[basura]

    print("Log-Rank Test para la combinación", valor)
    print("Estadístico:", result.test_statistic)
    print("Valor p:", result.p_value)
    print()
    basura = basura + 1

Log-Rank Test para la combinación ('C', 'CI')
Estadístico: 1631.4570136675688
Valor p: 0.0

Log-Rank Test para la combinación ('C', 'J')
Estadístico: 1938.793796601743
Valor p: 0.0

Log-Rank Test para la combinación ('C', 'JI')
Estadístico: 1938.793796601743
Valor p: 0.0

Log-Rank Test para la combinación ('C', 'SN')
Estadístico: 935.8640654236661
Valor p: 1.5688128536619692e-205

Log-Rank Test para la combinación ('CI', 'J')
Estadístico: 1949.124283359639
Valor p: 0.0

Log-Rank Test para la combinación ('CI', 'JI')
Estadístico: 1946.0440250109295
Valor p: 0.0

Log-Rank Test para la combinación ('CI', 'SN')
Estadístico: 955.7034051167933
Valor p: 7.637789138041729e-210

Log-Rank Test para la combinación ('J', 'JI')
Estadístico: 2008.051200332933
Valor p: 0.0

Log-Rank Test para la combinación ('J', 'SN')
Estadístico: 910.7720032998452
Valor p: 4.4681941186296475e-200

Log-Rank Test para la combinación ('JI', 'SN')
Estadístico: 906.386310915414
Valor p: 4.013459695590712e-199



In [ ]:


# Calcular el log-rank test
results = logrank_test(combinacion)

# Imprimir el p-valor del log-rank test
print("P-valor del log-rank test:", results.p_value)

In [9]:
import plotly.graph_objects as go
fig = go.Figure()
tratamientos = ["C","CI","J","JI","SN"]

for tratamiento in tratamientos:

    mi_db = bd6_V_farnesiana.loc[bd6_V_farnesiana['Tratamiento'] == tratamiento].copy()
    #print('Tratamiento:{}'.format(tratamiento))

    # Ajustar el modelo de Kaplan-Meier
    kmf = KaplanMeierFitter()
    kmf.fit(mi_db['Time'], mi_db['status'],label = "{}".format(tratamiento))

    # Imprimir la tabla de supervivencia
    #kmf.plot_survival_function(show_censors=True)
    #fig = px.line(kmf.survival_function_,title="Alturas H15")

    #fig.show()
    #print(kmf.survival_function_)
    fig.add_trace(go.Scatter(x=kmf.survival_function_.index, y=kmf.survival_function_["{}".format(tratamiento)],
                    mode='lines+markers',
                    name="{}".format(tratamiento)))
    #print(mi_db)
fig.update_layout(title='Prueba de super vivencia Kaplan-Meier')
fig.update_yaxes(
    range=(0, 1.1),
    constrain='domain'
)
fig.show()